# Feature Engineering - Simply Recipes Ingredients for CRF

In [1]:
import argparse
import nltk
import pandas as pd
import pycrfsuite
import numpy as np
import re
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/markishab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## NYT CRF MODEL

In [2]:
nyt_ing = pd.read_csv('../../data/01_raw/nyt-ingredients-snapshot-2015.csv')
nyt_ing.drop(columns=['index'], inplace=True)

In [3]:
nyt_ing.head()

,input,name,qty,range_end,unit,comment
0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,chopped coarse
4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,NaN


In [4]:
nyt_ing.fillna("missing", inplace=True)

Let's make ingredients list into tokens

In [5]:
ingredients_list = list(nyt_ing.input)
ingredients_list_new = []
for ingredient in ingredients_list:
    try: 
        ing_update = re.sub(r'(\d+)\s+(\d)/(\d)', r'\1$\2/\3', ingredient)
        ingredients_list_new.append(ing_update.split(" "))
    except:
        ingredients_list_new.append(np.nan)

Let's build a dictionary of the nyt dataset

In [6]:
# input_list = list(nyt_ing.input)
# name_list = list(nyt_ing.name)
# qty_list = list(nyt_ing.qty)
# unit_list = list(nyt_ing.unit)
# comment_list = list(nyt_ing.comment)

In [7]:
# # Let's add the dollar sign to the input list
# input_list_new = []
# for i in ingredients_list_new:
#     try:
#         input_list_new.append(re.sub(r'(\d+)\s+(\d)/(\d)', r'\1$\2/\3', inp))
#     except:
#         input_list_new.append(np.nan)

In [8]:
# def singularize(word):
#     """
#     A poor replacement for the pattern.en singularize function, but ok for now.
#     """

#     units = {
#         "cups": u"cup",
#         "tablespoons": u"tablespoon",
#         "teaspoons": u"teaspoon",
#         "pounds": u"pound",
#         "ounces": u"ounce",
#         "cloves": u"clove",
#         "sprigs": u"sprig",
#         "pinches": u"pinch",
#         "bunches": u"bunch",
#         "slices": u"slice",
#         "grams": u"gram",
#         "heads": u"head",
#         "quarts": u"quart",
#         "stalks": u"stalk",
#         "pints": u"pint",
#         "pieces": u"piece",
#         "sticks": u"stick",
#         "dashes": u"dash",
#         "fillets": u"fillet",
#         "cans": u"can",
#         "ears": u"ear",
#         "packages": u"package",
#         "strips": u"strip",
#         "bulbs": u"bulb",
#         "bottles": u"bottle"
#     }

#     if word in units.keys():
#         return units[word]
#     else:
#         return word

I can't make the name, quantity and comment match up with the right word. let's reevaluation what I'm doing. Do I really need to do this this way? 

In [9]:
# name_list_list = []
# for name in name_list:
#     name_list_list.append([name])

# qty_list_list = []
# for qty in qty_list:
#     qty_list_list.append([qty])

# unit_list_list = []
# for unit in unit_list:
#     unit_list_list.append([unit])

# comment_list_list = []
# for comment in comment_list:
#     comment_list_list.append([comment])

In [10]:
# for i in ingredients_list_new[1]:
#     for name, quantity, unit, comment in zip (name_list_list[1], qty_list_list[1], unit_list_list[1], comment_list_list[1]):
#         if singularize(i) in name:
#             label = 'name'
#         elif singularize(i) in unit:
#             label = 'unit'
#         elif singularize(i) in comment: 
#             label = 'comment'
#         else: 
#             label = 'qty'
#     print(i, label)

**The dictionary is missing at least 100K entries. What happened?**

In [11]:
# ing_dict = {}
# for inpu, name, qty, unt, comment in zip(input_list_new, name_list, qty_list, unit_list, comment_list):
#     ing_dict.update({inpu: {'name':name, 'qty':qty, 'unit':unt, 'comment':comment}})

In [12]:
# len(ing_dict.keys())

Let's add the full ingredient to the front of the list of tokens

In [13]:
# len(nyt_ing)

In [14]:
# len(input_list_new)

---

### let's try and make a tuple list from this dataframe instead

In [15]:
nyt_ing.head()

,input,name,qty,range_end,unit,comment
0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,"1 medium-size onion, peeled and chopped",onion,1.00,0,missing,"medium-size, peeled and chopped"
3,"2 stalks celery, chopped coarse",celery,2.00,0,stalk,chopped coarse
4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0,tablespoon,missing


In [16]:
nyt_ing_copy = nyt_ing.copy()

In [17]:
tuple_test = []
for index, row in nyt_ing.iterrows():
    name = row[1]
    qty = row[2]
    unit = row[4]
    comment = row[5]
    tuple_test.append([(name, 'name'), (qty, 'qty'), (unit, 'unit'), (comment, 'comment')])

In [29]:
tuple_test

[[('butternut squash', 'name'),
  (1.25, 'qty'),
  ('cup', 'unit'),
  ('cooked and pureed fresh, or 1 10-ounce package frozen squash, defrosted',
   'comment')],
 [('chestnuts', 'name'),
  (1.0, 'qty'),
  ('cup', 'unit'),
  ('peeled and cooked fresh (about 20), or 1 cup canned, unsweetened',
   'comment')],
 [('onion', 'name'),
  (1.0, 'qty'),
  ('missing', 'unit'),
  ('medium-size, peeled and chopped', 'comment')],
 [('celery', 'name'),
  (2.0, 'qty'),
  ('stalk', 'unit'),
  ('chopped coarse', 'comment')],
 [('vegetable oil', 'name'),
  (1.5, 'qty'),
  ('tablespoon', 'unit'),
  ('missing', 'comment')],
 [('water', 'name'), (0.5, 'qty'), ('cup', 'unit'), ('missing', 'comment')],
 [('gelatin', 'name'),
  (2.0, 'qty'),
  ('tablespoon', 'unit'),
  ('unflavored, dissolved in 1/2 cup water', 'comment')],
 [('Salt', 'name'), (0.0, 'qty'), ('missing', 'unit'), ('missing', 'comment')],
 [('plum tomatoes', 'name'),
  (1.0, 'qty'),
  ('cup', 'unit'),
  ('canned, with juice', 'comment')],
 [('sto

In [18]:
# tuple_test = []
# for index, row in nyt_ing.iterrows():
#     name = row[1].split(" ")
#     qty = row[2]
#     unit = row[4]
#     comment = str(row[5]).split(" ")
#     tuple_test.append([name, 'name', qty, 'qty', unit, 'unit', comment, 'comment'])

In [19]:
# new_tuple_list = []
# for i in tuple_test[0:2]:
#     tuple_sublist = []
#     for elem in i: 
#         print(elem)
#             if " " not in 
#         try:
#             if len(elem[0]) > 1:
#                 tuple_sublist.append((elem[0][0], elem[1]))
#                 tuple_sublist.append((elem[0][1], elem[1]))
#             else:
#                 tuple_sublist.append((elem[0], elem[1]))
                
#     new_tuple_list.append(tuple_sublist)
            

In [75]:
test = [[('butternut squash', 'name'),
  (1.25, 'qty'),
  ('cup', 'unit'),
  ('cooked and pureed fresh, or 1 10-ounce package frozen squash, defrosted',
   'comment')],
 [('chestnuts', 'name'),
  (1.0, 'qty'),
  ('cup', 'unit'),
  ('peeled and cooked fresh (about 20), or 1 cup canned, unsweetened',
   'comment')],
 [('onion', 'name'),
  (1.0, 'qty'),
  ('missing', 'unit'),
  ('medium-size, peeled and chopped', 'comment')],
 [('celery', 'name'),
  (2.0, 'qty'),
  ('stalk', 'unit'),
  ('chopped coarse', 'comment')],
 [('vegetable oil', 'name'),
  (1.5, 'qty'),
  ('tablespoon', 'unit'),
  ('missing', 'comment')],
 [('water', 'name'), (0.5, 'qty'), ('cup', 'unit'), ('missing', 'comment')],
 [('gelatin', 'name'),
  (2.0, 'qty'),
  ('tablespoon', 'unit'),
  ('unflavored, dissolved in 1/2 cup water', 'comment')],
 [('Salt', 'name'), (0.0, 'qty'), ('missing', 'unit'), ('missing', 'comment')],
 [('plum tomatoes', 'name'),
  (1.0, 'qty'),
  ('cup', 'unit'),
  ('canned, with juice', 'comment')],
 [('stock', 'name'),
  (6.0, 'qty'),
  ('cup', 'unit'),
  ('veal or beef', 'comment')],
 [('Worcestershire sauce', 'name'),
  (0.33, 'qty'),
  ('cup', 'unit'),
  ('missing', 'comment')],
 [('Louisiana hot sauce', 'name'),
  (1.0, 'qty'),
  ('tablespoon', 'unit'),
  ('missing', 'comment')],
 [('hot red pepper flakes', 'name'),
  (0.5, 'qty'),
  ('teaspoon', 'unit'),
  ('missing', 'comment')]]

In [82]:
ls = []
for sub_list in test: 
    sub_ls = []
    for elem in sub_list:
        if " " in str(elem[0]):
            elem2 = elem[0].split(" ")
            for el in elem2:
                sub_ls.append((el, elem[1]))
        else: 
            sub_ls.append((elem[0], elem[1]))
    ls.append(sub_ls)
            
    
# " " in test[0][0][0]

In [84]:
len(ls)

13

---

## CRF TUTORIAL: Performing Sequence Labelling using CRF in Python

In [20]:
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
import codecs

# Read data file and parse the XML
with codecs.open("../../data/tutorials/reuters.xml", "r", "utf-8") as infile:
    soup = bs(infile, "html5lib")

### Put Data in form that CRF likes it

In [21]:
docs = []
for elem in soup.find_all("document"):
    texts = []

    # Loop through each child of the element under "textwithnamedentities"
    for c in elem.find("textwithnamedentities").children:
        if type(c) == Tag:
            if c.name == "namedentityintext":
                label = "N"  # part of a named entity
            else:
                label = "I"  # irrelevant word
            for w in c.text.split(" "):
                if len(w) > 0:
                    texts.append((w, label))
    docs.append(texts)

In [22]:
docs

[[('Paxar', 'N'),
  ('Corp', 'N'),
  ('said', 'I'),
  ('it', 'I'),
  ('has', 'I'),
  ('acquired', 'I'),
  ('Thermo-Print', 'N'),
  ('GmbH', 'N'),
  ('of', 'I'),
  ('Lohn', 'N'),
  (',', 'I'),
  ('West', 'N'),
  ('Germany', 'N'),
  (',', 'I'),
  ('a', 'I'),
  ('distributor', 'I'),
  ('of', 'I'),
  ('Paxar', 'N'),
  ('products,', 'I'),
  ('for', 'I'),
  ('undisclosed', 'I'),
  ('terms.', 'I')],
 [('Key', 'N'),
  ('Tronic', 'N'),
  ('corp', 'N'),
  ('said', 'I'),
  ('it', 'I'),
  ('has', 'I'),
  ('received', 'I'),
  ('contracts', 'I'),
  ('to', 'I'),
  ('provide', 'I'),
  ('seven', 'I'),
  ('original', 'I'),
  ('equipment', 'I'),
  ('manufacturers', 'I'),
  ('with', 'I'),
  ('which', 'I'),
  ('it', 'I'),
  ('has', 'I'),
  ('not', 'I'),
  ('done', 'I'),
  ('business', 'I'),
  ('recently', 'I'),
  ('with', 'I'),
  ('over', 'I'),
  ('300,000', 'I'),
  ('computer', 'I'),
  ('keyboards', 'I'),
  ('for', 'I'),
  ('delivery', 'I'),
  ('within', 'I'),
  ('the', 'I'),
  ('next', 'I'),
  ('12', 'I'

In [23]:
data = []
for i, doc in enumerate(docs):

    # Obtain the list of tokens in the document
    tokens = [t for t, label in doc]
    
    # Perform POS tagging
    tagged = nltk.pos_tag(tokens)

    # Take the word, POS tag, and its label
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

### Construct Features

In [24]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [25]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

In [26]:
X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 13343
Seconds required: 0.062

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 5422.458004
Feature norm: 1.000000
Error norm: 5902.107019
Active features: 12911
Line search trials: 1
Line search step: 0.000042
Seconds required for this iteration: 0.007

***** Iteration #2 *****
Loss: 4406.313366
Feature norm: 0.849692
Error norm: 5304.568909
Active features: 13013
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.005

***** Iteration #3 *****
Loss: 4008.113407
Feature norm: 0.810059
Error norm: 12381.574837
Active features: 8581
Line search trials: 2
Line search step: 0.500000
Seconds required for this 

***** Iteration #58 *****
Loss: 214.156766
Feature norm: 43.150573
Error norm: 8.845258
Active features: 1999
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #59 *****
Loss: 214.022530
Feature norm: 43.149694
Error norm: 13.581895
Active features: 1984
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #60 *****
Loss: 213.860678
Feature norm: 43.161334
Error norm: 8.011241
Active features: 1970
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #61 *****
Loss: 213.728762
Feature norm: 43.161287
Error norm: 13.055320
Active features: 1961
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #62 *****
Loss: 213.582228
Feature norm: 43.147742
Error norm: 10.908313
Active features: 1935
Line search trials: 1
Line search step: 1.000000
Seconds required for this 

***** Iteration #142 *****
Loss: 210.061684
Feature norm: 43.733836
Error norm: 8.943046
Active features: 1748
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #143 *****
Loss: 210.037469
Feature norm: 43.736245
Error norm: 3.424842
Active features: 1744
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #144 *****
Loss: 210.037385
Feature norm: 43.736620
Error norm: 10.615596
Active features: 1743
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #145 *****
Loss: 210.006879
Feature norm: 43.741354
Error norm: 4.504084
Active features: 1744
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #146 *****
Loss: 210.005489
Feature norm: 43.741986
Error norm: 10.837799
Active features: 1742
Line search trials: 1
Line search step: 1.000000
Seconds required for t

***** Iteration #196 *****
Loss: 209.375313
Feature norm: 43.957785
Error norm: 2.856732
Active features: 1704
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.008

***** Iteration #197 *****
Loss: 209.370775
Feature norm: 43.956687
Error norm: 6.415370
Active features: 1696
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.006

***** Iteration #198 *****
Loss: 209.360131
Feature norm: 43.959878
Error norm: 7.047412
Active features: 1692
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #199 *****
Loss: 209.344535
Feature norm: 43.954382
Error norm: 4.464012
Active features: 1692
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.004

***** Iteration #200 *****
Loss: 209.336996
Feature norm: 43.954055
Error norm: 4.798822
Active features: 1694
Line search trials: 1
Line search step: 1.000000
Seconds required for thi

In [28]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

key (N)
tronic (N)
corp (N)
said (I)
it (I)
has (I)
received (I)
contracts (I)
to (I)
provide (I)
seven (I)
original (I)
equipment (I)
manufacturers (I)
with (I)
which (I)
it (I)
has (I)
not (I)
done (I)
business (I)
recently (I)
with (I)
over (I)
300,000 (I)
computer (I)
keyboards (I)
for (I)
delivery (I)
within (I)
the (I)
next (I)
12 (I)
months. (I)
the (I)
company (I)
said (I)
the (I)
new (I)
contracts (I)
represent (I)
an (I)
annual (I)
increase (I)
of (I)
approximately (I)
25 (I)
pct (I)
in (I)
unit (I)
volume (I)
over (I)
last (I)
year. (I)


**SOURCE:**
* <font color='red'>Performing Sequence Labelling using CRF in Python</font>
* https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html
* https://www.nltk.org/api/nltk.tag.html#module-nltk.tag.crf
* http://www.nltk.org/book/ch00.html
* https://python-crfsuite.readthedocs.io/en/latest/
* https://open.nytimes.com/
* CRF Suite Tutorial: http://www.chokkan.org/software/crfsuite/tutorial.html
* sklearn_crfsuite tutorial: https://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html
* NYT Ingredients Parser: https://github.com/nytimes/ingredient-phrase-tagger
* https://github.com/kulsoom-abdullah/kulsoom-abdullah.github.io/tree/master/AWS-lambda-implementation
* End to End Recipe Cuisine Classification: https://towardsdatascience.com/https-towardsdatascience-com-end-to-end-recipe-cuisine-classification-e97f4ac22104
* Performing Sequence Labelling using CRF in Python: http://www.albertauyeung.com/post/python-sequence-labelling-with-crf/